In [1]:
import pandas as pd
import pymongo
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# browser = init_browser()
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
INFO:WDM:Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
INFO:WDM:Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\Danny Dao\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
INFO:WDM:Driver [C:\Users\Danny Dao\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


### NASA Mars News

In [3]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, "html.parser")
# mars_news = soup.select_one("ul.item_list li.slide")
mars_news = soup.find('li', class_="slide")

In [5]:
# # mars_news.find("div", class_="content_title")
# news_title = mars_news.find('div', class_='content_title').text.strip()
# news_title
mars_news.find("div", class_="content_title")

<div class="content_title"><a href="/news/9211/nasa-mars-orbiter-releasing-one-of-its-last-rainbow-colored-maps/" target="_self">NASA Mars Orbiter Releasing One of Its Last Rainbow-Colored Maps </a></div>

In [6]:
news_title = mars_news.find("div", class_="content_title").get_text()
print(news_title)

NASA Mars Orbiter Releasing One of Its Last Rainbow-Colored Maps 


In [7]:
news_p = mars_news.find("div", class_="article_teaser_body").get_text()
print(news_p)

The map, to be released in batches over six months, covers the vast majority of the planet, revealing dozens of minerals found on its surface.


### Mars Space Images—Featured Image

In [8]:
#URL for Featured Space Image site
url_2 = "https://spaceimages-mars.com/"
browser.visit(url_2)

In [9]:
time.sleep(2)
full_image_button = browser.links.find_by_partial_text("FULL IMAGE").click()
# full_image_button.click()
# browser.click_link_by_partial_text('FULL IMAGE')


In [10]:
html = browser.html
image_soup = bs(html, "html.parser")

In [11]:
# Isolate the image url image source string by .find() and calling the ['src']
img = image_soup.find('img', class_='headerimage fade-in')

img_link =img['src']
img_link

'image/featured/mars1.jpg'

In [12]:
# Assign the string compound image url to a variable
featured_image_url = url_2 + img_link
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


### Mars Facts

In [13]:
#Visit the astrogeology site to obtain high-resolution images for each hemisphere of Mars.
# Define the url
url_3 = 'https://galaxyfacts-mars.com/'

# Using pd.read_html() will pull a list dataframes of all the tables
tables = pd.read_html(url_3)

# We want to slice off the 1st table from the list
mars_facts_tbl = tables[0]

# Name the columns
mars_facts_tbl.columns = ['Description', 'Mars', 'Earth']

# Set the index to the Atrributes column
mars_facts_tbl.set_index('Description', inplace=True)
mars_facts_tbl

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
#Use Pandas to convert the data to a HTML table string.
html_mars_tbl = mars_facts_tbl.to_html()
html_mars_tbl

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

### Mars Hemispheres

In [15]:
# Define URL
url_4= ('https://marshemispheres.com/')
    
# Use splinter browser to open url site
browser.visit(url_4)

#soupify
soup = bs(browser.html, 'html.parser')
    

In [16]:
# Create an empty list to store dictonary values for the keys of 'image_url' and 'title'
hemisphere_image_urls = []

# Use .find_all() to slice out the html we will loop through to visit different webpages and scrape the data
image_links = soup.find_all('div', class_='item')

# Start the for loop
for item in image_links:
    
    # Find the url link string from the 'a' tag and call the 'href' string
    link = item.find('a')['href']
    
    # Combine the root url from above and the link url
    url_5 = url_4 + link
    
    # Open the splinter browser using the url_4 link we just created
    browser.visit(url_5)
    
    # Let the browser load for 1 seconds before scraping data
    time.sleep(1)
    
    # Soupify the page
    soup = bs(browser.html, 'html.parser')
    
    # Find the link to the image in the 'ul' tag, then the 'a' tag, and then call the 2nd item 'href'
    # Store link string in variable 'image_link_hemi'
    image_link_hemi = soup.find('ul').find_all('a')[0]['href']
    
    # Find the title name using the 'h2' tag and class attribute 'title', and then pull the .text
    # Store title in variable 'title_text'
    title_text = soup.find('h2', class_='title').text
    
    # Append the hemisphere list with a dictionary of the keys and values
    hemisphere_image_urls.append({
        'title': title_text, 
        'img_url': image_link_hemi
    })
    
    # Print out success message
    print(f'Scrape of {title_text} COMPLETE')
    time.sleep(1)

Scrape of Cerberus Hemisphere Enhanced COMPLETE
Scrape of Schiaparelli Hemisphere Enhanced COMPLETE
Scrape of Syrtis Major Hemisphere Enhanced COMPLETE
Scrape of Valles Marineris Hemisphere Enhanced COMPLETE


In [17]:
# Check to see if data is all there
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'images/valles_marineris_enhanced-full.jpg'}]

In [18]:
# Create a dictionary of all the web scraped data
dict_mars_scrape = {
    'news_title': news_title,
    'news_p': news_p,
    'featured_image_url': featured_image_url,
    'html_mars_tbl': html_mars_tbl,
    'hemisphere_image_urls': hemisphere_image_urls
}
dict_mars_scrape

{'news_title': 'NASA Mars Orbiter Releasing One of Its Last Rainbow-Colored Maps ',
 'news_p': 'The map, to be released in batches over six months, covers the vast majority of the planet, revealing dozens of minerals found on its surface.',
 'featured_image_url': 'https://spaceimages-mars.com/image/featured/mars1.jpg',
 'html_mars_tbl': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>